<a href="https://colab.research.google.com/github/lmcanavals/aai/blob/main/ACO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import graphviz as gv
import numpy as np
import random
import math

In [2]:
class Ant:
    def __init__(self, numberOfCities):
        self.trailSize = numberOfCities
        self.trail = np.zeros((numberOfCities), dtype=int)
        self.visited = np.zeros((numberOfCities), dtype=int)

    def clear(self):
        for i in range(len(self.visited)):
            self.visited[i] = False

    def visitCity(self, currentIndex, city):
        self.trail[currentIndex + 1] = city
        self.visited[city] = True

    def trailLength(self, graph):
        length = graph[self.trail[self.trailSize - 1], self.trail[0]]
        for i in range(self.trailSize - 1):
            length += graph[self.trail[i], self.trail[i + 1]]
        return length

In [3]:
c = 1
alpha = 1
beta = 5
evaporation = 0.5
Q = 500
antFactor = 0.8
randomFactor = 0.01
maxIterations = 10

In [4]:
currentIndex = 0
numberOfCities = 15
graph = np.random.rand(numberOfCities, numberOfCities)
numberOfAnts = int(numberOfCities * antFactor)

trails = np.zeros((numberOfCities, numberOfCities))
probabilities = np.zeros((numberOfCities))
ants = [Ant(numberOfCities) for i in range(numberOfAnts)]

first = True
bestTourOrder = None
bestTourLength = math.inf

In [5]:
def setupAnts():
    for ant in ants:
        ant.clear()
        ant.visitCity(-1, random.randint(0, numberOfCities - 1))

    global currentIndex
    currentIndex = 0

$$
pheromone = \sum_{l: l \notin visited}{trail_{i,l}^\alpha * w_{i, l}^\beta}
$$
$$
prob(j) =  \frac{trail_{i, j}^\alpha * w_{i, j}^\beta}{pheromone}
$$

In [6]:
def calculateProbabilities(ant):
    global currentIndex
    i = ant.trail[currentIndex]
    pheromone = 0
    for l in range(numberOfCities):
        if not ant.visited[l]:
            pheromone += trails[i, l]**alpha * (1/graph[i, l])**beta
    for j in range(numberOfCities):
        if ant.visited[j]:
            probabilities[j] = 0
        else:
            numerator = trails[i, j]**alpha * (1/graph[i, j])**beta
            probabilities[j] = numerator / pheromone

In [7]:
def selectNextCity(ant):
    global currentIndex
    t = random.randint(0, numberOfCities - currentIndex - 1)
    if random.random() < randomFactor:
        indexes = np.nonzero(ant.visited)
        if t in indexes[0]:
            return t
    calculateProbabilities(ant)
    r = random.random()
    total = 0
    for i in range(numberOfCities):
        total += probabilities[i]
        if total >= r:
            return i

    return -1

In [8]:
def moveAnts():
    global currentIndex
    for i in range(currentIndex, numberOfCities - 1):
        for ant in ants:
            ant.visitCity(currentIndex, selectNextCity(ant))
        currentIndex += 1

In [9]:
def updateTrails():
    global currentIndex
    for i in range(numberOfCities):
        for j in range(numberOfCities):
            trails[i, j] *= evaporation

    for ant in ants:
        contribution = Q / ant.trailLength(graph)
        for i in range(numberOfCities - 1):
            trails[ant.trail[i], ant.trail[i+1]] += contribution

    trails[ant.trail[numberOfCities - 1], ant.trail[0]] += contribution

In [10]:
def updateBest():
    global first
    global bestTourLength
    global bestTourOrder
    if (first):
        first = False
        bestTourOrder = ants[0].trail
        bestTourLength = ants[0].trailLength(graph)
    for ant in ants:
        antTL = ant.trailLength(graph)
        if antTL < bestTourLength:
            bestTourLength = antTL
            bestTourOrder = ant.trail[:]

In [11]:
def clearTrails():
    for i in range(numberOfCities):
        for j in range(numberOfCities):
            trails[i, j] = c

In [12]:
def solve():
    setupAnts()
    clearTrails()
    for i in range(maxIterations):
        moveAnts()
        updateTrails()
        updateBest()
    print(f"Best tour length: {bestTourLength}")
    print(f"Best tour order: {bestTourOrder}")
    return bestTourOrder[:]

In [13]:
solve()

Best tour length: 1.7841746046237503
Best tour order: [12  3  2  5 14  7 13  4  0 10  6  8 11  1  9]


array([12,  3,  2,  5, 14,  7, 13,  4,  0, 10,  6,  8, 11,  1,  9])